In [9]:
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')

### Step 1: Splitting Shapefile into Counties

In [9]:
# script from chapter 7, pg 130, rewriten so as to work for this assignment.

import arcpy
C_source = r"F:\UW\GEOG458 Python\Arclab\Workspace\saep_bg10.shp"                  
C_output = r"F:\UW\GEOG458 Python\Arclab\Workspace\COUNTY_001.shp"
county = "COUNTYFP10"
delimfield = arcpy.AddFieldDelimiters(C_source, county)
arcpy.Select_analysis(C_source, C_output, delimfield + " = '001'")

<Result 'F:\\UW\\GEOG458 Python\\Arclab\\Workspace\\COUNTY_001.shp'>

###### 1.31.2016:
    
I went through a variety of methodoligies when trying to automate the above script. First I thought I could copy the scritp and do a line for each one so I ran a script to say what the county codes were After realizing there would be a lot of county codes, I then tried to make a loop to automate inserting a new county code into the select analysis as it came across it. Unfortunetly the script refuse to read the code.

In [3]:
# After fighting with trying to automate the above script for aproximately five hours over the weekend and two on monday, 
# I decided to take a look at what tool ArcMap had that could accomplish what I wanted to do.
# By using a mixture of ArcMap and ArcMap help, I decided to try Split Analysis.

import arcpy

arcpy.env.workspace=r"F:\UW\GEOG458 Python\Arclab\Workspace"    # Because workspace.                  
C_output = r"F:\UW\GEOG458 Python\Arclab\Workspace\scriptsplit" # C_source was not needed, workspace already set the source.
shapey = "saep_bg10.shp"
county = "COUNTYFP10"

arcpy.Split_analysis (shapey, shapey, county, C_output) #I used ArcMap and Help file to know what fields needed to be filled.
                            # I origionaly used the shapefile twice so as to get split analysis to use information in the same file.
print arcpy.GetMessages()   # From chapter 5.10, page 113 of the book. 
                            # I had hoped it would show the process as it was working, 
                            # but it did not give any of the messages until after all of the scripts were done running.

Executing: Split saep_bg10.shp washington.shp COUNTYFP10 "F:\UW\GEOG458 Python\Arclab\Workspace\scriptsplit" #
Start Time: Tue Feb 02 09:30:52 2016
Reading Features...
Cracking Features...
Assembling Features...
Succeeded at Tue Feb 02 09:33:00 2016 (Elapsed Time: 2 minutes 8 seconds)


> There is a divergence in date because I re-ran the above script multiple times to get 'clean' files to work with while 
trying to delete and organize the output county shapefiles in step 2. I initially tried to make a copy of the saep_bg10.shp 
file in hopes that it would remove the result of the script making duplicate files, but that did not work. So instead I decided 
to have a separate script that would delete the additional files.

###### 2.1.2016:  

I proceeded to try to find a way to write a script that would list all the files in the folder where 
the shape files had been split off to, and have a python list with those shapefile names.

Mid processes, I spoke with Hardy Wooldridge who mentioned that what was wanted, was a file that had all of the county names, not just the county codes. He also mentioned about the process of writing out what you want to accomplish with the code, and what tools could be used in ArcMap so as to achieve what was wanted. While I had already used the Split analysis, this did mean that there were tools to merge the information regarding the population in the shape files to total population. 

I decided to re-run the split analysis, after merging the County Names so as to not have to write more code then was needed.

In [10]:
import arcpy

# Setting Workspace
arcpy.env.workspace=r"F:\UW\GEOG458 Python\Arclab\Workspace"    # Because workspace.                  
C_output = r"F:\UW\GEOG458 Python\Arclab\Workspace" # C_source was not needed, workspace already set the source.

tablefile = "WashingtonFIPS.dbf"
tablefield = "FIPSCounty"
shapefile = "saep_bg10.shp"
shapefield = "COUNTYFP10"
outfeature = "countyjoin.shp"

arcpy.MakeFeatureLayer_management (shapefile, outfeature)  # Added to solve the AddJoin 'ERROR 000840' regarding shapefile type.
arcpy.AddJoin_management(outfeature, shapefield, tablefile, tablefield) # I reversed the order in which I joined the fields 
                                                                        # after multipule errors with joining the database 
                                                                        # with the shapefile.

<Result 'countyjoin.shp'>

While using a mix of ArcMap and the help files, I tried to get the Addjoin_management script to work for three hours without success. 

###### 2.2.2016:  

Following day I met with  Wooldridge who explained that the order in which I was joining layers were backwards, and that the file needed to be a certain type, which was why the script was not accepting the data I was trying to use. I then reversed the order, and used a script to turn the shapefile being used into a layer.

I then focused on organizing files by population typ

In [11]:
# listing the results from the created file.

fieldlist = arcpy.ListFields(outfeature, "", "String")
for field in fieldlist:
    print field.name + " " + str(field.length)
    
# Script added after having difficulty figuring out the name of the 
# "CountyName" field that would be used for the split analysis.

saep_bg10.STATEFP10 2
saep_bg10.COUNTYFP10 3
saep_bg10.TRACTCE10 6
saep_bg10.BLKGRPCE10 1
saep_bg10.GEOID10 12
saep_bg10.NAMELSAD10 13
saep_bg10.MTFCC10 5
saep_bg10.FUNCSTAT10 1
saep_bg10.Version 20
WashingtonFIPS.CountyName 254
WashingtonFIPS.FIPSCounty 254


After testing the split counties, I discovered that the split analysis tool was leaving out counties in the split. I therefore needed to try running a seperate script.

In [19]:
# Getting values of the county shapefile.
arcpy.env.workspace=r"F:\UW\GEOG458 Python\Arclab\Workspace"    # Because workspace.                  
C_output = r"F:\UW\GEOG458 Python\Arclab\Workspace\countysplit" 
county = "countyjoin.shp"
name = "WashingtonFIPS.CountyName"

values = [row[0] for row in arcpy.da.SearchCursor(county, (name))] #searches for values in 'name' from the 'county' table.
countyvalues = set(values)  # creates a new list 'countyvalue' with the values that the cursor found in 'name'
print(countyvalues)

set([u'San Juan', u'Pend Oreille', u'Snohomish', u'Grant', u'Pierce', u'Douglas', u'Kittitas', u'Jefferson', u'Whatcom', u'Wahkiakum', u'Cowlitz', u'Columbia', u'Thurston', u'Yakima', u'Island', u'Okanogan', u'Kitsap', u'Clallam', u'King', u'Lincoln', u'Spokane', u'Adams', u'Grays Harbor', u'Klickitat', u'Benton', u'Franklin', u'Ferry', u'Mason', u'Whitman', u'Skagit', u'Skamania', u'Clark', u'Stevens', u'Lewis', u'Walla Walla', u'Pacific', u'Garfield', u'Chelan', u'Asotin'])


In [38]:
# Re-Run split analysis with new script, this time using a loop that utilizes the value list just created.

arcpy.env.workspace=r"F:\UW\GEOG458 Python\Arclab\Workspace"                   
C_output = r"F:\UW\GEOG458 Python\Arclab\Workspace\thirdsplit" 
county = "countyjoin.shp"
cname = "WashingtonFIPS.CountyName"

for value in countyvalues:
        arcpy.SelectLayerByAttribute_management (county, "", '('cname') = ('value')')
        arcpy.CopyFeatures_management(county, C_output + value)
        print arcpy.GetMessages()   

SyntaxError: invalid syntax (<ipython-input-38-c14d2df2ae18>, line 9)

I fought with the value list for about two hours but kept recieving a SyntaxError when it came to selecting the attributes.  


>  File "<ipython-input-38-c14d2df2ae18>", line 9  
    arcpy.SelectLayerByAttribute_management (county, "", '('cname') = ('value')')  
                                                                ^
SyntaxError: invalid syntax  

2/2/2016: 3:36.
I put in a help request online to the TA. While waiting for a responce, I decided to use what I had and try to get a population count, and return to the SyntaxError issue later.

In [6]:
# A new split script.

arcpy.env.workspace=r"F:\UW\GEOG458 Python\Arclab\Workspace"    # Because workspace.                  
C_output = r"F:\UW\GEOG458 Python\Arclab\Workspace\countysplit" 
county = "countyjoin.shp"
name = "WashingtonFIPS.CountyName"
         
print arcpy.GetMessages()  

Executing: Split countyjoin.shp countyjoin.shp WashingtonFIPS.CountyName "F:\UW\GEOG458 Python\Arclab\Workspace\countysplit" #
Start Time: Tue Feb 02 10:54:50 2016
Reading Features...
Cracking Features...
Assembling Features...
Succeeded at Tue Feb 02 10:55:43 2016 (Elapsed Time: 53.07 seconds)


### Step 2: Calculating County Populations

In [2]:
# Getting list of shapefiles.

import arcpy
arcpy.env.workspace=r"F:\UW\GEOG458 Python\Arclab\Workspace\countysplit"

countlist = arcpy.ListFiles ("*.shp") 
print countlist

# I was able to get it to list all items but I needed to delete the duplicate shapefiles that the Split Analysis created.

[u'Adams.shp', u'Adams_.shp', u'Asotin.shp', u'Asotin_.shp', u'Benton.shp', u'Benton_.shp', u'Chelan.shp', u'Chelan_.shp', u'Clallam.shp', u'Clallam_.shp', u'Clark.shp', u'Clark_.shp', u'Columbia.shp', u'Columbia_.shp', u'Cowlitz.shp', u'Cowlitz_.shp', u'Douglas.shp', u'Douglas_.shp', u'Ferry.shp', u'Ferry_.shp', u'Franklin.shp', u'Franklin_.shp', u'Garfield.shp', u'Garfield_.shp', u'Grant.shp', u'Grant_.shp', u'Grays Harbor.shp', u'Grays Harbor_.shp', u'Island.shp', u'Island_.shp', u'Jefferson.shp', u'Jefferson_.shp', u'King.shp', u'King_.shp', u'Kitsap.shp', u'Kitsap_.shp', u'Kittitas.shp', u'Kittitas_.shp', u'Klickitat.shp', u'Klickitat_.shp', u'Lewis.shp', u'Lewis_.shp', u'Lincoln.shp', u'Lincoln_.shp', u'Mason.shp', u'Mason_.shp', u'Okanogan.shp', u'Okanogan_.shp', u'Pacific.shp', u'Pacific_.shp', u'Pend Oreille.shp', u'Pend Oreille_.shp', u'Pierce.shp', u'Pierce_.shp', u'San Juan.shp', u'San Juan_.shp', u'Skagit.shp', u'Skagit_.shp', u'Skamania.shp', u'Skamania_.shp', u'Snohomish

In [7]:
# Deleting Duplicates
import arcpy
from arcpy import env

env.workspace = r"F:\UW\GEOG458 Python\Arclab\Workspace\countysplit"
arcpy.management.Delete( * + '_.shp')
print arcpy.GetMessages()

TypeError: bad operand type for unary +: 'str'

In [8]:
# Deleting Duplicates
import arcpy
from arcpy import env

env.workspace = r"F:\UW\GEOG458 Python\Arclab\Workspace\countysplit"
arcpy.Delete_management('*_.shp')
print arcpy.GetMessages()

Executing: Delete *_.shp #
Start Time: Tue Feb 02 14:22:25 2016
WARNING 000110: *_.shp does not exist
Succeeded at Tue Feb 02 14:22:25 2016 (Elapsed Time: 0.00 seconds)


In [8]:
# Getting list of shapefiles.

import arcpy
arcpy.env.workspace=r"F:\UW\GEOG458 Python\Arclab\Workspace\countysplit"

countlist = arcpy.ListFiles ("*.shp") 
print countlist

# I was able to get it to list all items but I needed to delete the duplicate shapefiles that the Split Analysis created.

[u'Adams.shp', u'Adams_.shp', u'Asotin.shp', u'Asotin_.shp', u'Benton.shp', u'Benton_.shp', u'Chelan.shp', u'Chelan_.shp', u'Clallam.shp', u'Clallam_.shp', u'Clark.shp', u'Clark_.shp', u'Columbia.shp', u'Columbia_.shp', u'Cowlitz.shp', u'Cowlitz_.shp', u'Douglas.shp', u'Douglas_.shp', u'Ferry.shp', u'Ferry_.shp', u'Franklin.shp', u'Franklin_.shp', u'Garfield.shp', u'Garfield_.shp', u'Grant.shp', u'Grant_.shp', u'Grays Harbor.shp', u'Grays Harbor_.shp', u'Island.shp', u'Island_.shp', u'Jefferson.shp', u'Jefferson_.shp', u'King.shp', u'King_.shp', u'Kitsap.shp', u'Kitsap_.shp', u'Kittitas.shp', u'Kittitas_.shp', u'Klickitat.shp', u'Klickitat_.shp', u'Lewis.shp', u'Lewis_.shp', u'Lincoln.shp', u'Lincoln_.shp', u'Mason.shp', u'Mason_.shp', u'Okanogan.shp', u'Okanogan_.shp', u'Pacific.shp', u'Pacific_.shp', u'Pend Oreille.shp', u'Pend Oreille_.shp', u'Pierce.shp', u'Pierce_.shp', u'San Juan.shp', u'San Juan_.shp', u'Skagit.shp', u'Skagit_.shp', u'Skamania.shp', u'Skamania_.shp', u'Snohomish

In [ ]:
# I tried to run a simple python script that would delete a file.
import os
outSHPS= r"F:\UW\GEOG458 Python\Arclab\Workspace\countysplit" 
print 'removing duplicate shapefiles'

os.path.isdir(outSHPS):
    files = os.listdir(outSHPS)
    for f in files:
      full_path = os.path.join(outSHPS, f)
      if not os.path.isdir(full_path):
        os.remove(full_path)
os.remove('*_.shp')
print("File Removed!")
# I reworked the python script a few times but could not get it to delete the duplicate files I wanted it to.

In [ ]:
#  Getting population in shapefiles.
#  I decided to try to use dictionary files.

for count in countlist:
        arcpy.management.SelectLayerByAttribute(yourIntersectionsLayer,"NEW_SELECTION","\"ObjectID\" = "+str(row.ObjectID))  
            if x < 0:
                eventCursor = arcpy.SearchCursor(eventsLayer,"\"Year\" = 2007")
                for event in eventCursor:
                    year3+=1
                del event,eventCursor